In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stat

import warnings
warnings.simplefilter("ignore")

In [19]:
df = pd.read_excel('race_chien.xlsx',index_col=0)
df.head()

,Taille,Poids,Velocite,Intelligence,Affection,Agressivite,Fonction,Cote
Chien,,,,,,,,
Beauceron,Taille++,Poids+,Veloc++,Intell+,Affec+,Agress+,utilite,2.5
Basset,Taille-,Poids-,Veloc-,Intell-,Affec-,Agress+,chasse,4.5
Berger All,Taille++,Poids+,Veloc++,Intell++,Affec+,Agress+,utilite,3.0
Boxer,Taille+,Poids+,Veloc+,Intell+,Affec+,Agress+,compagnie,2.0
Bull-Dog,Taille-,Poids-,Veloc-,Intell+,Affec+,Agress-,compagnie,4.5


In [20]:
df.shape

(27, 8)

In [21]:
I, J = df.shape[0], df.shape[1]
print('Total Individus : {}\nTotal Variables : {}'.format(I, J))

Total Individus : 27
Total Variables : 8


In [22]:
# Statistiques descriptives
df.describe(include=[object])

,Taille,Poids,Velocite,Intelligence,Affection,Agressivite,Fonction
count,27,27,27,27,27,27,27
unique,3,3,3,3,2,2,3
top,Taille++,Poids+,Veloc-,Intell+,Affec+,Agress-,compagnie
freq,15,14,10,13,14,14,10


In [24]:
# for i,name in enumerate(df.columns):
#     plt.subplot(2, 3, i+1)
#     df[name].value_counts().plot.bar(figsize=(12,8))
#     plt.title(name)
#     plt.xticks(rotation= 0)
#     plt.tight_layout()
#     plt.suptitle("Diagramme en barres", fontsize=12)
# plt.show()

***Etude du croisement des variables prises 2 à 2 (Test du Khi-deux d'indépendance avec sa pvalue)***

In [35]:
chi2 = pd.DataFrame(np.zeros(shape=(J,J),dtype=float),index=df.columns, columns = df.columns)
pvalue = pd.DataFrame(np.zeros(shape=(J,J),dtype=float),index=df.columns, columns = df.columns)
for i in range(J):
    for j in range(i+1,J):
        tab = pd.crosstab(df.iloc[:,i],df.iloc[:,j])
        chi = stat.chi2_contingency(tab)
        chi2.iloc[i,j] = chi[0]
        pvalue.iloc[i,j]=chi[1]

print('------------- KHI-2 --------------')
chi2.round(3)

------------- KHI-2 --------------


,Taille,Poids,Velocite,Intelligence,Affection,Agressivite,Fonction,Cote
Taille,0.0,25.329,15.891,3.608,13.954,2.051,16.354,9.051
Poids,0.0,0.000,18.470,1.356,9.476,2.552,24.407,25.182
Velocite,0.0,0.000,0.000,3.156,2.967,0.575,8.483,13.390
Intelligence,0.0,0.000,0.000,0.000,3.891,1.157,4.140,15.779
Affection,0.0,0.000,0.000,0.000,0.000,0.915,14.761,5.370
Agressivite,0.0,0.000,0.000,0.000,0.000,0.000,7.073,9.576
Fonction,0.0,0.000,0.000,0.000,0.000,0.000,0.000,16.395
Cote,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [36]:
print('------------- P-Value --------------')
pvalue.round(3)

------------- P-Value --------------


,Taille,Poids,Velocite,Intelligence,Affection,Agressivite,Fonction,Cote
Taille,0.0,0.0,0.003,0.462,0.001,0.359,0.003,0.527
Poids,0.0,0.0,0.001,0.852,0.009,0.279,0.000,0.005
Velocite,0.0,0.0,0.000,0.532,0.227,0.750,0.075,0.203
Intelligence,0.0,0.0,0.000,0.000,0.143,0.561,0.387,0.106
Affection,0.0,0.0,0.000,0.000,0.000,0.339,0.001,0.372
Agressivite,0.0,0.0,0.000,0.000,0.000,0.000,0.029,0.088
Fonction,0.0,0.0,0.000,0.000,0.000,0.000,0.000,0.089
Cote,0.0,0.0,0.000,0.000,0.000,0.000,0.000,0.000


*Les variables **taille, poids et vélocité** sont très liés entre eux. Les variables **intelligence et agressivité sont indépendantes entre elles et des autres variables**. L’ACM va permettre, entre autres, de retrouver ces configurations types en visualisant les races canines et les modalités.*

**Conception du tableau disjonctif complet (TDC)**

Dans le TDC, les lignes sont les individus et les colonnes sont les modalités des variables qualitatives

In [40]:
dummies = pd.get_dummies(df, prefix="",prefix_sep="")
dummies.head(2)

,Cote,Taille+,Taille++,Taille-,Poids+,Poids++,Poids-,Veloc+,Veloc++,Veloc-,Intell+,Intell++,Intell-,Affec+,Affec-,Agress+,Agress-,chasse,compagnie,utilite
Chien,,,,,,,,,,,,,,,,,,,,
Beauceron,2.5,0,1,0,1,0,0,0,1,0,1,0,0,1,0,1,0,0,0,1
Basset,4.5,0,0,1,0,0,1,0,0,1,0,0,1,0,1,1,0,1,0,0


L'ACM va étudier la variabilité des individus, c'est-à-dire les ressemblances et différences entre individus <br>
Pour ce faire, l'ACM va extraire les différentes dimensions de variabilités avant de les décrire par les variables (modalités).<br>
<br>
Du point des varaibles, l'ACM va s'interesser aux associations entre modalités